### DTIHAN 

In [17]:
import torch as t
import dgl

import numpy as np
import pandas as pd
import scipy as sp
import scipy.io as sio

from random import sample
from scipy import linalg
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import KFold  # k-folds CV

from utils import EarlyStopping
from model_hetero import HAN

In [27]:
# HAN embedding 函数

def DTI_embedding(param):
    # param 参数字典

    # 解析参数
    dgc = param.get('dgc')
    dc = param.get('dc')
    drugAttribute = param.get('drugAttribute')
    dg = param.get('dg')
    targetAttribute = param.get('targetAttribute')

    drug_max_iters = param.get('drug_max_iters')
    target_max_iters = param.get('target_max_iters')
    
    drug_hidden_size = param.get('drug_hidden_size')
    drug_out_size = param.get('drug_out_size')
    drug_num_heads = param.get('drug_num_heads')
    drug_dropout = param.get('drug_dropout')
    drug_label = param.get('drug_label')

    target_hidden_size = param.get('target_hidden_size')
    target_out_size = param.get('target_out_size')
    target_num_heads = param.get('target_num_heads')
    target_dropout = param.get('target_dropout')
    target_label = param.get('target_label')

    path_to_csv = param.get('path_to_csv')

    # 源节点 -> 目标节点
    begin_dgc,end_dgc=[],[]
    for i in range(np.shape(dgc)[0]):
        for j in range(np.shape(dgc)[1]):
            if dgc[i][j]==1:
                begin_dgc.append(j)
                end_dgc.append(i)

    # 药物，边
    begin_dc,end_dc,sim_dc=[],[],[]
    drug_edge_dict = {}
    edge, edge_index = None, None
    for i in range(np.shape(dc)[0]):
        for j in range(0,np.shape(dc)[1]):
            begin_dc.append(i)
            end_dc.append(j)
            edge=str(i)+'=>'+str(j)
            edge_index=len(sim_dc)
            sim_dc.append(dc[i][j])
            drug_edge_dict[edge] = edge_index

    #创建异构图drug_target_1来获得药物隐特征，选取药物-靶标-药物和药物-药物-药物作为元路径
    drug_target_1 = dgl.heterograph({
         ('drug','dt','target'):(begin_dgc,end_dgc),
         ('target','td','drug'):(end_dgc,begin_dgc),
         ('drug','dd','drug'):(begin_dc,end_dc),
         ('drug','dd_r','drug'):(end_dc,begin_dc),
    })
    sim_dc=np.array(sim_dc)
    #加入药物间的相似度特征
    drug_target_1.edges['dd'].data['sim']=t.from_numpy(sim_dc)
    #输入药物的初始节点特征
    drug_target_1.nodes['drug'].data['feature']=t.from_numpy(drugAttribute).float()

    # 靶标，边
    begin_dg,end_dg,sim_dg = [],[],[]
    target_edge_dict = {}
    edge, edge_index = None, None
    for i in range(np.shape(dg)[0]):
        for j in range(0,np.shape(dg)[1]):
            begin_dg.append(i)
            end_dg.append(j)
            edge=str(i)+'=>'+str(j)
            edge_index=len(sim_dg)
            sim_dg.append(dg[i][j])
            target_edge_dict[edge] = edge_index

    #创建异构图drug_target_2来获得靶标隐特征，选取靶标-药物-靶标和靶标-靶标-靶标作为元路径
    drug_target_2 = dgl.heterograph({
         ('target','td','drug'):(end_dgc,begin_dgc),
         ('drug','dt','target'):(begin_dgc,end_dgc),
         ('target','tt','target'):(begin_dg,end_dg),
         ('target','tt_r','target'):(end_dg,begin_dg),
    })
    sim_dg=np.array(sim_dg)
    #加入靶标间的相似度特征
    drug_target_2.edges['tt'].data['sim']=t.from_numpy(sim_dg)
    #输入靶标的初始节点特征
    drug_target_2.nodes['target'].data['feature']=t.from_numpy(targetAttribute).float()

    # 药物网络训练
    drug_features = drug_target_1.nodes['drug'].data['feature'].to('cuda:0')
    drug_model = HAN(meta_paths=[['dt','td'],['dd','dd_r']],
                     in_size=drug_features.shape[1],
                     hidden_size=drug_hidden_size,
                     out_size=drug_out_size,
                     num_heads=drug_num_heads,
                     dropout=drug_dropout).to('cuda:0')
    drug_target_1 = drug_target_1.to('cuda:0')
    # stopper = EarlyStopping(patience=100)
    loss_fcn = t.nn.MSELoss()
    optimizer = t.optim.Adam(drug_model.parameters(), lr=0.006,
                                     weight_decay=0.001)
    for epoch in range(drug_max_iters):
        drug_model.train()
        logits = drug_model(drug_target_1, drug_features)
        loss = loss_fcn(drug_target_1.edges['dd'].data['sim'].float(), (logits@logits.T).view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print('Epoch {:d} | Train Loss {:.4f}'.format(epoch+1,loss.item()))
    
    logits = logits.cpu().detach().numpy()
    # 保存数据
    drug_result = pd.DataFrame(logits)
    drug_result.index = drug_label

    #获取靶标隐特征
    #特征维度=hidden_size*num_heads
    print('target embedding...')
    target_features = drug_target_2.nodes['target'].data['feature'].to('cuda:0')
    target_model = HAN(meta_paths=[['td','dt'],['tt','tt_r']],
                       in_size=target_features.shape[1],
                       hidden_size=target_hidden_size,
                       out_size=target_out_size,
                       num_heads=target_num_heads,
                       dropout=target_dropout).to('cuda:0')
    drug_target_2 = drug_target_2.to('cuda:0')
    # stopper = EarlyStopping(patience=100)
    loss_fcn = t.nn.MSELoss()
    optimizer = t.optim.Adam(target_model.parameters(), lr=0.006,
                                     weight_decay=0.001)
    
    for epoch in range(target_max_iters):
        target_model.train()
        logits = target_model(drug_target_2, target_features)
        loss = loss_fcn(drug_target_2.edges['tt'].data['sim'].float(), (logits@logits.T).view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print('Epoch {:d} | Train Loss {:.4f}'.format(epoch+1,loss.item()))
        
    logits = logits.cpu().detach().numpy()
    # 保存数据
    target_result = pd.DataFrame(logits)
    target_result.index = target_label

    # 输出保存
    df = pd.concat([target_result, drug_result], axis=0)
    # out_result = target_result.append(drug_result)
    df.to_csv(path_to_csv)

In [19]:
# 评价函数
# 计算 ROC 的 AUC 值
def AUCValue(label, score):
    label = np.copy(label)
    score = np.copy(score)
    fpr, tpr, thresholds = roc_curve(label, score, pos_label=1)
    roc_auc = auc(fpr, tpr)
    return roc_auc

# 计算 PR 的 AUC 值
def PRAUCValue(label, score):
    label = np.copy(label)
    score = np.copy(score)
    pr_auc = average_precision_score(label, score)
    return pr_auc

In [20]:
# FPRM class
class FPRM:
    '''
    类属性：
    特征投影矩阵 R
    正则化参数 re_lambda

    类成员：（尽量与 sklearn 一致）
    训练函数 fit
    预测函数 predict_pair
    预测函数 predict_matrix
    '''

    # 初始化函数
    def __init__(self,
                 re_lambda = None):
        self.R = None
        self.re_lambda = re_lambda

    # 计算矩阵逆，如果矩阵不可逆，求其最小二乘逆
    def inverse_matrix(self,X):
        X_inv = None
        try:
            X_inv = linalg.inv(X)
        except:
            print('sigular matrix, compute linalg.lstsq solve inv(X)')
            n,m = X.shape
            Ematrix = np.eye(n,n,dtype=float)
            X_inv = linalg.lstsq(X,Ematrix)[0]
        finally:
            return X_inv

    # 训练函数
    def fit(self,X,Y,Delta):
        '''

        :param X:  特征矩阵，一行一个样本, N * m, N 个样本，m 个特征
        :param Y:  特征矩阵，一行一个样本， N * n, N 个样本，n 个特征
        :param Delta: 对应着特征矩阵 X,Y 的关系作用矩阵，维度 N * N，x_i R y_j^T = delta_{ij}
        :return: None
        '''

        x = np.copy(X)
        y = np.copy(Y)
        delta = np.copy(Delta)

        xTx = x.T.dot(x)   # x^T * x
        yTy = y.T.dot(y)   # y^T * y
        xTDy = x.T.dot(delta).dot(y)
        if self.re_lambda is None:
            # 没有正则项
            xTx_inv = self.inverse_matrix(xTx)
            yTy_inv = self.inverse_matrix(yTy)
            self.R = xTx_inv.dot(xTDy).dot(yTy_inv)
        else:
            # 没有正则项，需要求解 Sylvester 方程
            yTy_inv = self.inverse_matrix(yTy)
            B = self.re_lambda * yTy_inv
            Q = xTDy.dot(yTy_inv)
            # 注意，scipy.linalg.solve_sylvester 在 scipy 1.1.0 版本之后
            self.R = linalg.solve_sylvester(a=xTx,b=B,q=Q)

        return self


    # 预测函数，矩阵形式
    def predict_matrix(self,X,Y):
        '''

        :param X: 特征矩阵，一行一个样本
        :param Y: 特征矩阵，一行一个样本， X,Y 样本个数必须一致，特征维度可以不一致
        :return: 预测关系矩阵
        '''
        x = np.copy(X)
        y = np.copy(Y)

        return x.dot(self.R).dot(y.T)

    # 预测函数，pair 形式
    def predict_pair(self,X,Y):
        '''

        :param X: 特征矩阵，一行一个样本
        :param Y: 特征矩阵，一行一个样本， X,Y 样本个数必须一致，特征维度可以不一致
        :return: 返回对应样本数的向量
        '''
        n,m = X.shape
        val = np.zeros((n,1),dtype=float)
        for i in range(n):
            val[i] = X[i].dot(self.R).dot(Y[i].T)

        return val


# 特征投影关系隶属度类，继承 FPRM 类
# 会新添一些属性：
# ulabel  唯一标签 list
# dict_label  唯一标签，dict
# mu,sigma  隶属度函数参数，每一个标签对应一组参数
class FPRMmembership(FPRM):
    # 初始化函数
    def __init__(self,
                 re_lambda=None):
        FPRM.__init__(self, re_lambda=re_lambda)
        # 下面是新增属性
        self.ulabel = None     # 唯一标签，list
        self.dict_label = None    # 唯一标签，dict
        self.mu = None         # 对应类别标签的隶属度函数均值参数, dict 保存，易于参数值与标签对应
        self.sigma2 = None     # 对应类别标签的隶属度函数方差参数，dict 保存，易于参数值域标签对应

    # 训练函数
    def fit(self,X,Y,Delta):
        '''

        :param X:  特征矩阵，一行一个样本, N * m, N 个样本，m 个特征
        :param Y:  特征矩阵，一行一个样本， N * n, N 个样本，n 个特征
        :param Delta: 对应着特征矩阵 X,Y 的关系作用矩阵，维度 N * N，x_i R y_j^T = delta_{ij}
        :return: None
        '''

        delta = np.array(Delta)
        # 矩阵中不重复元素
        delta_list = delta.flatten()
        ulabel = np.unique(delta_list).tolist()   # 转成 list 形式

        # 计算 R
        FPRM.fit(self,X=X,Y=Y,Delta=Delta)
        # 计算训练集的预测值
        pred_delta = FPRM.predict_matrix(self,X=X,Y=Y)

        # 构造标签的索引字典，方便读取索引值
        D = {}
        mu = {}
        sigma2 = {}
        for i,val in enumerate(ulabel):
            D[val] = i      # 标签值为 val 对应 ulabel 的索引为 i, 即 ulabel[i] = val
            # 计算均值与方差
            mu[val] = np.mean(pred_delta[ Delta == val])
            sigma2[val] = np.var(pred_delta[Delta == val])
            # print('label : %s , mean : %.4f , var : %.4f' %(val,mu[val],sigma2[val]))

        self.ulabel = ulabel
        self.dict_label = D
        self.mu = mu
        self.sigma2 = sigma2
        return self

    # 预测函数，矩阵形式，返回指定标签的隶属度值
    def predict_score_matrix(self,X,Y,label):
        '''

        :param X: 特征矩阵，一行一个样本
        :param Y: 特征矩阵，一行一个样本， X,Y 样本个数必须一致，特征维度可以不一致
        :param label: 需要计算隶属度的标签
        :return:
        '''

        predict_value = X.dot(self.R).dot(Y.T)
        if label in self.ulabel:
            val2 = np.square(predict_value - self.mu[label])
            membership = np.exp(- val2 / (2.0 * self.sigma2[label]))
            return membership
        else:
            print('interaciton label error!')
            exit()


    #预测函数，pair 形式，返回指定标签的隶属度值
    def predict_score_pair(self,X,Y,label):
        '''

        :param X: 特征矩阵，一行一个样本
        :param Y: 特征矩阵，一行一个样本， X,Y 样本个数必须一致，特征维度可以不一致
        :param label: 指定需要返回标签的隶属度
        :return:
        '''

        n = X.shape[0]
        val = np.zeros((n,1),dtype=float)
        for i in range(n):
            val[i] = X[i].dot(self.R).dot(Y[i].T)

        if label in self.ulabel:
            val2 = np.square(val - self.mu[label])
            membership = np.exp(- val2 / (2.0 * self.sigma2[label]))
            return membership
        else:
            print('interaction label error!')
            exit()

In [21]:
# 交叉验证

def get_k_folds(tsize,kfolds=10):
    '''

    :param tsize:  需要交叉验证的样本总数
    :param kfolds:
    :return:
    '''

    indices = np.array(range(tsize))
    np.random.shuffle(indices)
    testFolds = []
    trainFolds = []
    skf = KFold(n_splits=kfolds)
    for trIndex, teIndex in skf.split(indices):
        Xtr,Xte = indices[trIndex], indices[teIndex]
        testFolds.append(Xte.tolist())
        trainFolds.append(Xtr.tolist())

    return testFolds,trainFolds

# 用于计算 AUC, AUPR 的交叉验证函数，确保交叉后每一行、每一列至少一个非 0
def doCrossVal(inMat,kfold=10,numSplit=5):
    '''
    :param inMat: 相互作用矩阵，其值为 {0,1}
    :param kfold:  交叉折数
    :param numSplit:  重复次数
    :return:
    '''

    tempMat = np.array(np.copy(inMat))
    # deal with one position
    # all one position
    rowIndexs, colIndexs = np.where(tempMat == 1)
    df_onePos = pd.DataFrame({'rowIndexs': rowIndexs, 'colIndexs': colIndexs}, columns=['rowIndexs', 'colIndexs'])

    nLink = 1  # 至少需要满足 nLink+1 的作用关系
    df_row_size = df_onePos.groupby('rowIndexs')['colIndexs'].size()  # 按 rowIndexs 分组，统计属性 colIndexs 值的个数
    ind = df_row_size.index[df_row_size.values > nLink]
    df_onePos = df_onePos[df_onePos['rowIndexs'].isin(ind)]
    df_col_size = df_onePos.groupby('colIndexs')['rowIndexs'].size()  # 按 colIndexs 分组，size() 函数作用在 rowIndexs 属性上
    ind = df_col_size.index[df_col_size.values > nLink]
    df_onePos = df_onePos[df_onePos['colIndexs'].isin(ind)]

    # 为了确保能够进行安全的交叉验证，一个好的想法是：
    # 先把不可进行交叉的作用关系剔除（该行或列只有一个 1）（上述代码已经完成这项工作）
    # 其次，再给每行、列都随机保留一个 1，这样剩余的 1 进行交叉验证，无论如何划分，都可以确保每行、列至少有一个 1
    # randomly remove [one row], when 'col' > 1
    grouped_size = df_onePos.groupby(by='colIndexs')['rowIndexs'].size()
    dupCol = grouped_size.index[grouped_size.values > 1].values
    if len(dupCol) > 0:
        tmp = df_onePos[df_onePos['colIndexs'].isin(dupCol)]
        grouped_col = tmp.groupby(by='colIndexs')
        del_row_indexs = []
        del_col_indexs = []
        for name, group in grouped_col:
            del_col_indexs.append(int(name))
            del_row_indexs.append(sample(list(group['rowIndexs'].values), 1)[0])  # 随机的抽取一个
        df_del_indexs = pd.DataFrame({'rowIndexs': del_row_indexs, 'colIndexs': del_col_indexs},
                                     columns=['rowIndexs', 'colIndexs'])

        # 采用求差集的方式从 df_onePos 中删除 df_del_indexs
        # 由于 DataFrame 中没有求差集操作，先如下进行
        # df_del_indexs 已经全部在 df_onePos 中，并且 df_onePos 中并无重复元素，因此，可以把 df_del_indexs 拼接到 df_onePos 中
        # 然后，删除所有重复元素即可
        # 去重，利用drop_duplicates方法，a=a.drop_duplicates()，以及设置参数keep=False，意思就是只要有重复，重复的记录都去掉
        # df_onePos = df_onePos.append(df_del_indexs)  # 拼接
        df_onePos = pd.concat([df_onePos, df_del_indexs])  # 拼接
        df_onePos = df_onePos.drop_duplicates(keep=False)  # 去重，并且不保留重复记录

    # randomly remove [one col] , when 'row' > 1
    grouped_size = df_onePos.groupby(by='rowIndexs')['colIndexs'].size()
    dupRow = grouped_size.index[grouped_size.values > 1].values
    if len(dupRow) > 0:
        tmp = df_onePos[df_onePos['rowIndexs'].isin(dupRow)]
        grouped_row = tmp.groupby(by='rowIndexs')
        del_row_indexs = []
        del_col_indexs = []
        for name, group in grouped_row:
            del_row_indexs.append(int(name))
            del_col_indexs.append(sample(list(group['colIndexs'].values), 1)[0])  # 随机抽取一列

        df_del_indexs = pd.DataFrame({'rowIndexs': del_row_indexs, 'colIndexs': del_col_indexs},
                                    columns=['rowIndexs', 'colIndexs'])
        # 取 df_onePos - df_del_indexs 差集
        # df_onePos = df_onePos.append(df_del_indexs)  # 拼接
        df_onePos = pd.concat([df_onePos, df_del_indexs]) # 拼接
        df_onePos = df_onePos.drop_duplicates(keep=False)  # 去重，并且不保留重复记录

    # deal with zeros position
    rowIndexs, colIndexs = np.where(tempMat == 0)
    df_zerosPos = pd.DataFrame({'rowIndexs': rowIndexs, 'colIndexs': colIndexs}, columns=['rowIndexs', 'colIndexs'])

    # 检查是否剩余足够多正例进行交叉验证
    numOne = df_onePos.shape[0]  # 正例个数
    if numOne < kfold:
        print('available number of ONEs is less than %s' % kfold)
        sys.exit()
    numZero = df_zerosPos.shape[0]
    if numZero < kfold:
        print('available number of ZEROs is less than %s' % kfold)
        sys.exit()

    # 生成交叉验证数据集
    savedFolds = []  # 保存交叉数据集，长度等于重复的次数，每一个元素保存一个 nfold 折交叉验证
                     # 训练集保存将测试集对应作用关系置为 0 的作用矩阵
                     # 测试集保存为 DataFrame, 3 列 （rowIndexs,colIndexs,value）
    for i in range(numSplit):
        # 第 i 次重复
        list_i = []  # 保存第 i 个 split 的 nfold cv 数据
        # 只要挑出测试集，训练集只需要将原始作用矩阵对应的测试集为 1 的值置为 0 即可
        # 因此，可以不对无需对 get_k_folds 返回的训练集进行处理
        # 虽然 ONE, ZERO 分开处理的，但是对于 ZERO 部分，无需重新置 0，只需将 ONE 部分置 0
        # for ONEs
        test_one_Folds, trainFolds = get_k_folds(tsize=numOne, kfolds=kfold)
        # for ZEROs
        test_zero_Folds, trainFolds = get_k_folds(tsize=numZero, kfolds=kfold)
        for j in range(kfold):
            dict_j = {}  # 以字典形式保存第 j 折的数据
            # 测试集
            # for ONEs
            one_Index = test_one_Folds[j]
            one_rowIndexs = df_onePos.iloc[one_Index]['rowIndexs'].values
            one_colIndexs = df_onePos.iloc[one_Index]['colIndexs'].values
            one_testSet = df_onePos.iloc[one_Index][['rowIndexs', 'colIndexs']]  # DataFrame 格式
            one_testSet['value'] = 1  # 增加一列
            # for ZEROs
            # 由于 ZEROs 无需在 interaction 矩阵置 0 ，可以不用提取 row/col Index, 而是直接提取
            # 对应的 DataFrame
            zero_Index = test_zero_Folds[j]
            zero_testSet = df_zerosPos.iloc[zero_Index][['rowIndexs', 'colIndexs']]  # DataFrame 格式
            zero_testSet['value'] = 0  # 增加一列

            # one_testSet 与 zero_testSet 合并起来就是最终的测试集，且 DataFrame 格式
            # testSet = one_testSet.append(zero_testSet)
            testSet = pd.concat([one_testSet, zero_testSet], axis=0)
        

            # 训练集
            # 只需要在原始的 interaction 矩阵中，将 one_testSet 对应的 1 置为 0 即可
            tmp = np.copy(tempMat)  # data 为原始的作用矩阵，np.array 格式
            tmp[one_rowIndexs, one_colIndexs] = 0  # 置 0

            # 保存
            dict_j['testSet'] = testSet  # DataFrame   [rowIndexs,colIndexs,value]
            dict_j['foldMat'] = tmp  # np.array

            list_i.append(dict_j)

        savedFolds.append(list_i)

    return savedFolds

# cross validation funciton to calculate AUPR and AUC
def CV_AUPR_AUC(interactionMatrix,drugAttribute,targetAttribute,re_lambda,
                numSplit=3,kfolds=10):
    '''

    :param interactionMatrix:  drug-target interaction matrix, target * drug
    :param drugAttribute  drug 特征， drug 数 * 特征维数
    :param targetAttribute target 特征，target 数 * 特征维数
    :param re_lambda   FPRM 正则化参数
    :param numSplit: 重复次数
    :param kfolds:   交叉次数
    :return:
    '''

    inMat = np.copy(interactionMatrix)
    drug_attr = np.copy(drugAttribute)
    target_attr = np.copy(targetAttribute)

    # 交叉验证
    savedFolds = doCrossVal(inMat=inMat,kfold=kfolds,numSplit=numSplit)
    # 在 savedFolds 中的作用关系矩阵 foldMat 与输入关系矩阵 inMat 形式上一致
    # 由于在这里 inMat 输入时是 target * drug 矩阵，因此，foldMat 也是 target * drug 矩阵

    aupr_split = []
    auc_split = []
    for i,spliting in enumerate(savedFolds):
        aupr_cv = []
        auc_cv = []
        for j,folding in enumerate(spliting):
            # 准备数据集， train, test

            # train data
            # 交叉验证中已经返回了将测试集置 0 的作用矩阵，直接读取即可
            delta = folding['foldMat']       # target * drug 作用矩阵

            # FPRM 模型的预测得分 score
            clf = FPRMmembership(re_lambda=re_lambda)
            clf.fit(X=target_attr,Y=drug_attr,Delta=delta)
            score = clf.predict_score_matrix(X=target_attr,Y=drug_attr,label=1) # target * drug 得分矩阵
            #print(delta.shape,target_attr.shape,drug_attr.shape,score.shape)
            #print(delta)
            #print(score)
            #(26, 54) (26, 25) (54, 25) (26, 54)
            # AUPR、AUC 值
            # 提取出 testSet 中所有 score
            testSet = folding['testSet']   # DataFrame  ['rowIndexs','colIndexs','value']
            rowIndexs = testSet['rowIndexs'].values
            colIndexs = testSet['colIndexs'].values
            test_score = score[rowIndexs,colIndexs]
            test_label = testSet['value'].values
            
            aupr = PRAUCValue(label=test_label,score=test_score)
            aupr_cv.append(aupr)
            auc_val = AUCValue(label=test_label, score=test_score)
            auc_cv.append(auc_val)

        aupr_split.append(np.mean(aupr_cv))
        auc_split.append(np.mean(auc_cv))


    aupr_mean = np.mean(np.array(aupr_split))
    aupr_std = np.std(np.array(aupr_split))
    auc_mean = np.mean(np.array(auc_split))
    auc_std = np.std(np.array(auc_split))
    return (aupr_mean,aupr_std,auc_mean,auc_std,clf)

In [161]:
# 参数
curr_data_number = 3   # 当前数据集

# 药物网络参数
drug_hidden_sizes=[70,46,23,5]
drug_num_heads_array=[3,2,5,5]
drug_out_size = [16, 16, 16, 16]
drug_dropout = 0.4
# 靶标网络参数
target_hidden_sizes=[70,46,23,5]
target_num_heads_array=[3,2,5,5]
target_out_size = [16, 16, 16, 16] # out_size 对应 model_hetero 中 HAN 输出最后一个线性层，本代码没有采用该层，所以最终 embedding维度为 hidden_size * num_heads
target_dropout = 0.4
# 迭代次数
drug_max_iters = [300, 100, 200, 100]
target_max_iters = [800, 100, 300, 200]


# 用于 FPRM 的特征维度，与 embedding 维度保持一致
f_n_s = [210,92,115,25]  # 特征维度

# lambda_feature : 特征权重

lambda_features = [0.5, 0.2, 0.4, 0.01]


drug_f_n = f_n_s[curr_data_number]  # drug 特征维度
target_f_n = f_n_s[curr_data_number]  # target 特征维度

# 数据读取
# 1、路径与名称
path_embeding = 'result/'  # embedding 数据路径
path_mds = 'data/mds-mat/'  # mds 数据路径
path_original = 'data/original-mat/'  # 原始数据路径
name_mds = ['Enzyme', 'GPCR', 'IonChannel', 'NuclearRecept']   # mds 数据
name_embedding = ['e_result', 'gpcr_result', 'ic_result', 'nr_result']  # embedding 数据
name_interaction = ['AdmatEnzyme','AdmatGPCR','AdmatIonChannel','AdmatNuclearRecept']  # 原始数据集中相互作用矩阵
name_csim = ['CsimmatEnzyme','CsimmatGPCR','CsimmatIonChannel','CsimmatNuclearRecept']  # 原始数据集中药物-药物相似度
name_tsim = ['TsimmatEnzyme','TsimmatGPCR','TsimmatIonChannel','TsimmatNuclearRecept']  # 原始数据集中靶标-靶标相似度


# 2、mds 数据
# 读取数据
mds_data = sio.loadmat(path_mds+name_mds[curr_data_number]+'.mat')

# drug 特征， drug 数 * 特征维数, 序号对应 interaction 列顺序
drugAttribute = np.array(mds_data['drugAttribute'],dtype=float)
# target 特征， target 数 * 特征维数, 序号对应 interaction 行顺序
targetAttribute = np.array(mds_data['targetAttribute'],dtype=float)
# 对 drug, target 特征进行归一化
drug_clf = MinMaxScaler()
drugAttribute = drug_clf.fit_transform(drugAttribute)
target_clf = MinMaxScaler()
targetAttribute = target_clf.fit_transform(targetAttribute)
# ID
drug_ID = [x[0] for x in mds_data['drugID'][:,0]]
target_ID = [x[0] for x in mds_data['targetID'][:,0]]


# 3、interaction 数据从原始数据 original 中读取
# 读取数据
interaction_data = sio.loadmat(path_original+name_interaction[curr_data_number]+'.mat')
interaction_mat = np.array(interaction_data['matrix'], dtype=int)  # target * drug

# ID benchmark
drug_ID_benchmark = [x[0] for x in interaction_data['drugID'][:,0]]
target_ID_benchmark = [x[0] for x in interaction_data['targetID'][:,0]]

# 4、药物-药物、靶标-靶标相似度矩阵
csim_data = sio.loadmat(path_original+name_csim[curr_data_number]+'.mat')
csim_mat = np.array(csim_data['matrix'], dtype=float)  # drug * drug
tsim_data = sio.loadmat(path_original+name_tsim[curr_data_number]+'.mat')
tsim_mat = np.array(tsim_data['matrix'], dtype=float)  # target * target

# 5、药物、靶标嵌入数据准备
dgc = np.copy(interaction_mat)  # 互作矩阵
dc = np.copy(csim_mat)  # 药物相似度矩阵
dg = np.copy(tsim_mat)  # 靶标相似度矩阵

dc = (dc + dc.T) / 2.0
dg = (dg + dg.T) /2.0

drug_label = np.array(drug_ID)
target_label = np.array(target_ID)

In [162]:
param = {'drug_max_iters':drug_max_iters[curr_data_number],
          'target_max_iters':target_max_iters[curr_data_number],
         'dgc':dgc,
         'dc':dc,
         'drugAttribute':drugAttribute,
         'dg':dg,
         'targetAttribute':targetAttribute,
         'drug_hidden_size':drug_hidden_sizes[curr_data_number],
         'drug_out_size':drug_out_size[curr_data_number], 
         'drug_num_heads': [drug_num_heads_array[curr_data_number]],
         'drug_dropout':drug_dropout,
         'drug_label':drug_label,
         'target_hidden_size':target_hidden_sizes[curr_data_number],
         'target_out_size':target_out_size[curr_data_number],
         'target_num_heads':[target_num_heads_array[curr_data_number]],
         'target_dropout':target_dropout,
         'target_label':target_label,
         'path_to_csv': 'result/'+name_embedding[curr_data_number]+'.csv'}
DTI_embedding(param=param)

Epoch 1 | Train Loss 24.3996
Epoch 51 | Train Loss 0.0654
target embedding...
Epoch 1 | Train Loss 10.9080
Epoch 51 | Train Loss 0.0501
Epoch 101 | Train Loss 0.0334
Epoch 151 | Train Loss 0.0350


In [167]:
# FPRM 模型准备数据

# 读取 embedding 数据
embedding_data = pd.read_csv(path_embeding+name_embedding[curr_data_number]+'.csv', index_col=0, header=0)
# embedding_data.head(-5)
drug_embedding_df = embedding_data[embedding_data.index.str.contains('D')]
target_embedding_df = embedding_data[embedding_data.index.str.contains('hsa')]
# embedding_data.index.str.contains('hsa')  #  embedding_data.index 中包含 'hsa' 
# 对 drug, target 特征进行归一化
drug_clf = MinMaxScaler()
target_clf = MinMaxScaler()
# data to DataFrame, index 为 ID
drug_embedding_df = pd.DataFrame(data = drug_clf.fit_transform(drug_embedding_df.values), index=drug_embedding_df.index, columns=drug_embedding_df.columns)
target_embedding_df = pd.DataFrame(data = target_clf.fit_transform(target_embedding_df.values), index=target_embedding_df.index, columns=target_embedding_df.columns)

# 数据对齐
# data to DataFrame, index 为 ID
# 注意截取数据维度，与embedding维度保持一致
drug_mds_df = pd.DataFrame(data=drugAttribute[:,0:drug_f_n], index=drug_ID)
target_mds_df = pd.DataFrame(data=targetAttribute[:, 0:target_f_n], index=target_ID)
# mds, embedding 数据以 interaction 的 ID benchmark 为基准对齐，
# drug
temp_df = pd.DataFrame(data=drug_ID_benchmark, index=drug_ID_benchmark, columns=['ID'])
# mds 数据对齐
drug_mds_df = pd.merge(temp_df, drug_mds_df, how='left', left_index=True, right_index=True, sort=False)
drug_mds_df.drop(['ID'], axis=1, inplace=True)
# embedding 数据对齐
drug_embedding_df = pd.merge(temp_df, drug_embedding_df, how='left', left_index=True, right_index=True, sort=False)
drug_embedding_df.drop(['ID'], axis=1, inplace=True)
# target
temp_df = pd.DataFrame(data=target_ID_benchmark, index=target_ID_benchmark, columns=['ID'])
# mds 数据对齐
target_mds_df = pd.merge(temp_df, target_mds_df, how='left', left_index=True, right_index=True, sort=False)
target_mds_df.drop(['ID'], axis=1, inplace=True)
# embedding 数据对齐
target_embedding_df = pd.merge(temp_df, target_embedding_df, how='left', left_index=True, right_index=True, sort=False)
target_embedding_df.drop(['ID'], axis=1, inplace=True)



# 特征加权
# feature = lambda_feature * mds_feature + (1-lambda_feature) * embedding_feature
# drug
lambda_feature = lambda_features[curr_data_number]
# lambda_feature = 0.01
temp = (lambda_feature * drug_mds_df.values + (1.0-lambda_feature) * drug_embedding_df.values) / 2.0
drug_feature_df = pd.DataFrame(data=temp, index=drug_mds_df.index, columns=drug_mds_df.columns)
# target
temp = (lambda_feature * target_mds_df.values + (1.0-lambda_feature) * target_embedding_df.values) / 2.0
target_feature_df = pd.DataFrame(data=temp, index=target_mds_df.index, columns=target_mds_df.columns)


# DTI 预测验证
# 特别注意，由于整个模型存在局部最优，随机划分等因素，所以可以多重复运行几次，选择最优结果

print('数据集：{}'.format(name_mds[curr_data_number]))

# 正则化参数
lambdas = np.linspace(0.001, 0.20, 20)
# re_lambda = 1.0

# 重复次数，交叉验证次数
numSplit = 3
folds = 10
max_AUPR=0
max_AUC=0
for re_lambda in lambdas:
    # 交叉验证
    # interactionMatrix 需要形式  target * drug
    res = CV_AUPR_AUC(interactionMatrix=interaction_mat,
                      drugAttribute=drug_feature_df.values,
                      targetAttribute=target_feature_df.values,
                      re_lambda=re_lambda,
                      numSplit=numSplit,
                      kfolds=folds)
    
    if res[2]+res[3]>max_AUC:
        max_AUC=res[2]+res[3]
        optim_clf=res[4]
   
    print('lambda = {:.2f}, AUPR = {:.4f} +/- {:.4f}, AUC = {:.4f} +/- {:.4f}'.format(re_lambda, res[0], res[1], res[2], res[3]))




数据集：NuclearRecept
lambda = 0.00, AUPR = 0.5132 +/- 0.0148, AUC = 0.9021 +/- 0.0025
lambda = 0.01, AUPR = 0.5505 +/- 0.0151, AUC = 0.9582 +/- 0.0037
lambda = 0.02, AUPR = 0.4611 +/- 0.0514, AUC = 0.9538 +/- 0.0028
lambda = 0.03, AUPR = 0.6839 +/- 0.0431, AUC = 0.9495 +/- 0.0027
lambda = 0.04, AUPR = 0.5467 +/- 0.0641, AUC = 0.9567 +/- 0.0008
lambda = 0.05, AUPR = 0.5393 +/- 0.0174, AUC = 0.9470 +/- 0.0011
lambda = 0.06, AUPR = 0.5356 +/- 0.0781, AUC = 0.9469 +/- 0.0194
lambda = 0.07, AUPR = 0.6179 +/- 0.0535, AUC = 0.9612 +/- 0.0052
lambda = 0.08, AUPR = 0.5296 +/- 0.0285, AUC = 0.9543 +/- 0.0023
lambda = 0.10, AUPR = 0.5992 +/- 0.0187, AUC = 0.9465 +/- 0.0092
lambda = 0.11, AUPR = 0.4792 +/- 0.0381, AUC = 0.9257 +/- 0.0065
lambda = 0.12, AUPR = 0.5859 +/- 0.0401, AUC = 0.9494 +/- 0.0038
lambda = 0.13, AUPR = 0.3997 +/- 0.0105, AUC = 0.9156 +/- 0.0054
lambda = 0.14, AUPR = 0.4488 +/- 0.0321, AUC = 0.9550 +/- 0.0019
lambda = 0.15, AUPR = 0.5958 +/- 0.0212, AUC = 0.9706 +/- 0.0033
lambda 